> Currently i am writing this notebook my rank is `300` in MNIST with score of `.99567`9
16/10/20

# MNIST - Digit Recognition using Tensorflow

### Contents

* 1. Importing Libraries
* 2. Importing Dataset
    * 2.1 Histogram
    * 2.1 Plotting Figures
* 3. Data Preprocessing
    * 3.1 Converting Labels to Categorical
    * 3.2 Reshaping
    * 3.3 Data Normalization
    * 3.4 Spliting Data into Training Set & Validation Set
    * 3.5 Data Argumentation
* 4. CNN Model
    * 4.1 Creating Model
    * 4.2 Model Training
    * 4.3 Model Accuracy and Loss Plot
    * 4.4 Confusion Matrix and Classification Report
* 5. Final Prediction

# 1. Importing Libraries

In [ ]:
import numpy as np 
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.layers import Conv2D, BatchNormalization, Activation, MaxPooling2D, GlobalAveragePooling2D, Dense, Flatten, Dropout
from keras.optimizers import RMSprop, Adam, SGD
from keras import regularizers
from keras.callbacks import CSVLogger, ModelCheckpoint, ReduceLROnPlateau

# 2. Importing Dataset

In [ ]:
train = pd.read_csv('../input/digit-recognizer/train.csv')
test = pd.read_csv('../input/digit-recognizer/test.csv')
train.head(5)

In [ ]:
print('Train set: ',train.shape, "\t Test Set", test.shape)

So there are 42k images in training set and 28k images in testing set

In [ ]:
y = train['label']
# droping label cloumn in training set
train.drop('label', axis=1, inplace=True)

### 2.1 Histogram

In [ ]:
g = sns.countplot(y)

The plot shows the frequency of Labels, the most fequent label is 1

## 2.1 Plotting Figures

In [ ]:
plt.figure(figsize=(12,5))
for i in range(40):
    plt.subplot(4,10,i+1)
    img = train.iloc[i,:].values.reshape(28,28)
    plt.imshow(img)
    plt.axis('off')
plt.tight_layout()
plt.show()

# 3 Data Preprocessing

### 3.1 Converting Labels to Categorical

In [ ]:
from keras.utils.np_utils import to_categorical 
y = to_categorical(y, num_classes = 10)
y[0]

Labels are 10 digits numbers from 0 to 9. We need to encode these lables to one hot vectors (ex : 9 -> [0,0,0,0,0,0,0,0,0,1]).

### 3.2 Reshaping 

In [ ]:
train = train.values.reshape(train.shape[0], 28, 28, 1)
test = test.values.reshape(test.shape[0], 28, 28, 1)
print('Reshaped Train set: ',train.shape, " & Reshaped Test Set", test.shape)

MNIST images are gray scaled so it use only 1 channel, we would reshape the image to 28x28x1

### 3.3 Data Normalization

We have to perform normalization on  images [0..1] data than on [0..255]. to reduce the effect of illumination's differences.

The Normailzed data learns faster as there is less gradient to cover

In [ ]:
train = train.astype("float32")/255.0
test = test.astype("float32")/255.0

### 3.4 Spliting Data into Training Set & Validation Set
We have 42k images in training set we can split it into training and validation sets, so that we can be sure that the model is not ovefitted on training data. I have splites 20 % of data to validation set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train, y, test_size=0.25, random_state=0)

print("Number of samples in Training set :", X_train.shape[0])
print("Number of samples in Validation set :", X_val.shape[0])

### 3.5 Data Argumentation

In [ ]:
train_datagen = ImageDataGenerator(rotation_range=10,
                                   zoom_range=0.1,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1
                                  )

training_set = train_datagen.flow(X_train, y_train,
                                  batch_size=64
                                 )

val_datagen = ImageDataGenerator()
val_set = val_datagen.flow(X_val, y_val,
                           batch_size=64
                          )

* Randomly rotate some training images by 10 degrees
* Randomly Zoom by 10% some training images
* Randomly shift images horizontally by 10% of the width
* Randomly shift images vertically by 10% of the height

Since vertical_flip nor horizontal_flip could have lead to misclassify symetrical numbers such as 6 and 9, so i did not use it

# 4. CNN Model

### 4.1 Creating Model
I tried several model and this has given me the best accuracy so far

In [ ]:
model = tf.keras.models.Sequential()

model.add(Conv2D(64, kernel_size=(5,5), padding='same', activation='relu', input_shape=(28,28,1)))
model.add(Conv2D(64, kernel_size=(5,5), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128, kernel_size=(3,3), activation='relu', padding='same'))
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy']
             )
model.summary()

In [ ]:
# If the model is not improving on validation, we need to reduce the learning rate, If val loss is not improved in 4 epoch then lr will be reduced 
reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
                              factor=0.2, 
                              patience=4, 
                              verbose=1, 
                              min_delta=0.0001)

### 4.2 Model Training 

In [ ]:
steps_per_epoch = training_set.n // training_set.batch_size
validation_steps = val_set.n // val_set.batch_size

hist = model.fit(x=training_set,
                 validation_data=val_set,
                 epochs=35,
                 callbacks=[reduce_lr],
                 steps_per_epoch=steps_per_epoch,
                 validation_steps=validation_steps
                )

i might have run it more a bit longer i think 25 or 20 epochs will do fine

### 4.3 Model Accuracy and Loss Plot

In [ ]:
plt.figure(figsize=(14,5))
plt.subplot(1,2,2)
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['train', 'test'], loc='upper left')

plt.subplot(1,2,1)
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
_, acc_val = model.evaluate(val_set)
_, acc_tr = model.evaluate(val_set)
print("\nFinal Accuracy on training set : {:.2f}% & accuracy on validation is set: {:.2f}%".format(acc_tr*100, acc_val*100))

now the final accuracy of model on test and training set satisfactiory

In [ ]:
from keras.utils import plot_model
plot_model(model, show_shapes=True, show_layer_names=True)

### 4.4 Confusion Matrix and Classification Report

In [ ]:
val_pred = model.predict(val_set)
val_pred = np.argmax(val_pred, axis=1)
y_val = np.argmax(y_val, axis=1)

from sklearn.metrics import confusion_matrix, classification_report
print("Confusion Matrix")
cm = confusion_matrix(y_val, val_pred)
print(cm)
print("Classification Report")
print(classification_report(y_val, val_pred))

#g = sns.heatmap(cm, cmap='Blues')
plt.figure(figsize=(8,8))
plt.imshow(cm, interpolation='nearest')
plt.colorbar()
target_names = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
tick_mark = np.arange(len(target_names))
_ = plt.xticks(tick_mark, target_names)
_ = plt.yticks(tick_mark, target_names)

# 5. Final Prediction

In [ ]:
pred = model.predict(test)
res = np.argmax(pred, axis=1)
submission = pd.DataFrame({"ImageId":[i+1 for i in range(len(test))],
                           "Label": res})
submission.head(10)

In [ ]:
submission.to_csv("submission.csv", index=False)

* Your feedback in comments is much appreciated, Comment if you have any doubts or for inprovement
* Please UPVOTE if you LIKE this notebook, it will keep me motivated